# Prepare Data Sets

Prepare the necessary reference data set.

1. Natural products from ChEMBL (ChEMBL NPs)
2. Approved and experimental drugs from DrugBank (DrugBank)
3. Advanced Screening Collection (Enamine)

In [2]:
%reload_ext autoreload
%autoreload 2
# def warn(*args, **kwargs):
# warn    pass  # to silence scikit-learn warnings

import warnings
warnings.filterwarnings('ignore')
# warnings.warn = warn

# Global Imports
# from collections import Counter
# import glob
from pathlib import Path
import sys

import pandas as pd
import numpy as np
# from pandarallel import pandarallel
# pandarallel.initialize()

# from scipy.stats import median_absolute_deviation as mad

from rdkit import DataStructs
from rdkit.Chem import AllChem as Chem
# from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Project-local Imports
PROJECT_DIR = list(Path("..").absolute().parents)[1]
sys.path.append(str(PROJECT_DIR))
import plt_style
import utils as u
from utils import lp

Path("../tmp").mkdir(exist_ok=True)

## ChEMBL NP

Two versions of ChEMBL26 were downloaded from the FTP server (http://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/releases/chembl_26/) on 25-Mar-2020:

* `chembl_26_postgresql.tar.gz`
* `chembl_26.sdf.gz`

The PostgreSQL database was installed locally and all entries which had ("J. Nat. Prod" OR "J Nat Prod") as the journal source were extracted with the following SQL statement:

```
sql = """
select distinct(md.chembl_id) as Chembl_Id
from docs d,
     molecule_dictionary md,
     compound_records cr
where md.molregno = cr.molregno
  and cr.doc_id = d.doc_id
  and d.journal in ('J. Nat. Prod.', 'J Nat Prod')
"""
```

This list was saved as `chembl_26_natprot_ids_only.tsv` in the `Input Data` folder of this project and was used for filtering the ChEMBL_Ids from the SD file in the following procedure.

<hr>

* 40525 cpds.
* 35026 cpds. after deglycosylation, standardization and deduplication

### Loading

Load the result of the SQL query, the ChEMBL IDs that have "J. Nat. Prod." or "J Nat Prod" listed as journal:

In [10]:
chembl_np_ids = set(pd.read_csv("../Input Data/chembl_26_natprot_ids_only.tsv", sep="\t")["Chembl_Id"].values)
lp(chembl_np_ids)

Length                                            :      40547


Load the downloaded ChEMBL 26 SD file (gzipped) and keep only the natural product IDs:

In [12]:
# Takes 10min
chembl_np = u.read_sdf("/home/pahl/comas/notebooks/sdf/cpd_libs_2003/chembl_26.sdf.gz", merge_prop="chembl_id", merge_list=chembl_np_ids)

RDKit ERROR: [17:15:06] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [17:15:06] ERROR: Could not sanitize molecule ending on line 44486287
RDKit WARNING: [17:18:12] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:18:34] Warning: conflicting stereochemistry at atom 10 ignored. by rule 2b.
RDKit WARNING: [17:18:34] Warning: conflicting stereochemistry at atom 0 ignored. by rule 2b.
RDKit WARNING: [17:18:34] Warning: conflicting stereochemistry at atom 11 ignored. by rule 2b.
RDKit WARNING: [17:19:32] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:19:40] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:19:41] Warning: conflicting stereochemistry at atom 8 ignored. by rule 2b.
RDKit WARNING: [17:19:46] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:21:42] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:21:42] WARNING: not removing hydro

{'In': 1940733, 'Out': 40525, 'Fail_NoMol': 1, 'NotMerged': 1900207}


In [16]:
lp(chembl_np, "ChEMBL NP")
chembl_np.head()

Shape ChEMBL NP                                   :      40525 /    3  [ Smiles, Name, chembl_id ]  


,Smiles,Name,chembl_id
0,CC(=O)O[C@H]1/C(C)=C\[C@@H]2OC(=O)[C@]3(C)O[C@...,CHEMBL503865,CHEMBL503865
1,CO[C@H]1C[C@H](COC[C@H]2[C@@H](OC)C[C@H](O[C@H...,CHEMBL503873,CHEMBL503873
2,COc1ccc2c(c1OC)C(CC1(C)C=Cc3c(c4cccc(OC)c4n(C)...,CHEMBL504077,CHEMBL504077
3,CC(C)=C[C@H]1C[C@](C)(O)[C@@H]2[C@H]3CC[C@@H]4...,CHEMBL501665,CHEMBL501665
4,CCCCC[C@H]1CCCCCCCCCC(=O)O[C@@H]2[C@@H](O[C@@H...,CHEMBL501672,CHEMBL501672


### Initial Standardization

Standardization implemented in MolVS:
* standardization
* remove stereo information
* choose largest fragment

In [17]:
# Takes 3min
chembl_np = u.apply_to_smiles(chembl_np, "Smiles", {"Smiles_Std": lambda x: u.standardize_mol(x, remove_stereo=True)})
chembl_np.drop("Smiles", axis=1, inplace=True)
chembl_np = chembl_np.rename(columns={"Smiles_Std": "Smiles"})

RDKit ERROR: [17:31:36] ERROR: Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [18:19:29] Explicit valence for atom # 26 B, 5, is greater than permitted


In [18]:
lp(chembl_np, "ChEMBL NP")

Shape ChEMBL NP                                   :      40525 /    3  [ Name, chembl_id, Smiles ]  ( NAN values in 1 col(s) )


### Filters (MedChem, Duplicates)

The data set is filtered for Isotopes, MedChemAtoms, MinHeavyAtoms and Duplicates.  
The deduplication is performed here as well to remove as many compounds as possible at early stages of the processing.  
The filter for MaxHeavyAtoms is deferred until after the deglycosylation.

In [19]:
# Takes 2min
chembl_np = u.filter_mols(chembl_np, "Smiles", ["Isotopes", "MedChemAtoms", "MinHeavyAtoms", "Duplicates"])
lp(chembl_np, "ChEMBL NP (MedChem filters)")

chembl_np.head(10)

Applying filters (4)...



Applied filter Isotopes: 40518



Applied filter MedChemAtoms: 40481



Applied filter MinHeavyAtoms: 40478



Applied filter Duplicates: 36489
Shape ChEMBL NP (MedChem filters)                 :      36489 /    3  [ Name, chembl_id, Smiles ]  


,Name,chembl_id,Smiles
0,CHEMBL503865,CHEMBL503865,CC(=O)OC1C(C)=CC2OC(=O)C3(C)OC23C(OC(C)=O)C2C(...
1,CHEMBL503873,CHEMBL503873,COC1CC(COCC2C(C)OC(OC3CCC4(C)C(=CCC5(O)C4CC(OC...
2,CHEMBL504077,CHEMBL504077,COc1ccc2c(c1OC)C(CC1(C)C=Cc3c(c4cccc(OC)c4n(C)...
3,CHEMBL501665,CHEMBL501665,CC(C)=CC1CC(C)(O)C2C3CCC4C5(C)CCC(OC6OCC(O)C(O...
4,CHEMBL501672,CHEMBL501672,CCCCCC1CCCCCCCCCC(=O)OC2C(OC3OC(C)C(OC(=O)C(C)...
5,CHEMBL501674,CHEMBL501674,COC1C(OCC2C(C)OC(=O)C=CC(C)C(OC3OC(C)CC(O)C3O)...
6,CHEMBL501943,CHEMBL501943,COc1cc(C(O)C(COC(=O)C=Cc2ccc(O)cc2)Oc2c(OC)cc(...
7,CHEMBL501944,CHEMBL501944,COc1cc(O)c(-c2cc(-c3cc(-c4c(O)cc(O)c5c4CC(C)NC...
8,CHEMBL501918,CHEMBL501918,CC1COC2(CC1O)OC1CC3C4CC=C5CC(O)CC(OC6OCC(O)C(O...
9,CHEMBL501922,CHEMBL501922,COC(=O)CC1C(C)(C)C(OC(C)=O)C2C(=O)C1(C)C1CCC3(...


In [20]:
chembl_np.to_csv("../tmp/chembl_np.tsv", sep="\t", index=False)

### Deglycosylation

Deglycosylation is performed using the KNIME Sugar Remover node (CDK). KNIME v 4.2.3.  
Here, the result of the deglycosylation is loaded again.

In [2]:
chembl_np = pd.read_csv("../tmp/chembl_np_deglyco.tsv", sep="\t")
lp(chembl_np, "ChEMBL NP (Deglyco)")

Shape ChEMBL NP (Deglyco)                         :      34479 /    3  [ Name, chembl_id, Smiles ]  


### Standardization

Cleanup after deglycosylation:

* standardization
* choose largest fragment


In [3]:
# Takes 1:20min
chembl_np_std = u.apply_to_smiles(chembl_np, "Smiles", {"Smiles_Std": u.standardize_mol})
chembl_np_std.drop("Smiles", axis=1, inplace=True)
chembl_np_std = chembl_np_std.rename(columns={"Smiles_Std": "Smiles"})
lp(chembl_np_std, "ChEMBL NP (Deglyco)")


Shape ChEMBL NP (Deglyco)                         :      34479 /    3  [ Name, chembl_id, Smiles ]  


In [4]:
chembl_np_std = u.filter_mols(chembl_np_std, "Smiles", ["MinHeavyAtoms", "MaxHeavyAtoms", "Duplicates"])
lp(chembl_np_std, "ChEMBL NP (Filters)")

Applying filters (3)...



Applied filter MinHeavyAtoms: 34158
Applied filter MaxHeavyAtoms: 33676



Applied filter Duplicates: 30796
Shape ChEMBL NP (Filters)                         :      30796 /    3  [ Name, chembl_id, Smiles ]  


### Canonical Tautomer

In [5]:
# Takes 28min
chembl_np_std_tauto = u.apply_to_smiles(chembl_np_std, "Smiles", {"Smiles_Std": lambda x: u.standardize_mol(x, canonicalize_tautomer=True)}, parallel=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
chembl_np_std_tauto.drop("Smiles", axis=1, inplace=True)
chembl_np_std_tauto = chembl_np_std_tauto.rename(columns={"Smiles_Std": "Smiles"})

### Filter Duplicates

Remove duplicates that might have been formed by canonicalization.

In [7]:
chembl_np_std_tauto = u.filter_mols(chembl_np_std_tauto, "Smiles", "Duplicates")
lp(chembl_np_std_tauto, "ChEMBL NP (Filter duplicates)")

chembl_np_std_tauto.head()

Applying filters (1)...



Applied filter Duplicates: 30655
Shape ChEMBL NP (Filter duplicates)               :      30655 /    3  [ Name, chembl_id, Smiles ]  


,Name,chembl_id,Smiles
0,CHEMBL503865,CHEMBL503865,CC(=O)OC1/C(C)=C\C2OC(=O)C3(C)OC23C(OC(C)=O)C2...
1,CHEMBL503873,CHEMBL503873,CC(=O)C1(O)CCC2(O)C3(O)CC=C4CC(O)CCC4(C)C3CC(O...
2,CHEMBL504077,CHEMBL504077,COc1ccc2c(c1OC)C(CC1(C)C=Cc3c(c4cccc(OC)c4n(C)...
3,CHEMBL501665,CHEMBL501665,CC(C)=CC1CC(C)(O)C2C3CCC4C5(C)CCC(O)C(C)(C)C5C...
4,CHEMBL501672,CHEMBL501672,CCCCCC1CCCCCCCCCC(=O)OC2C(O)C(C)OC(OC3C(OC(CO)...


### Write Data Set

In [8]:
chembl_np_std_tauto.to_csv("../tmp/chembl_np_std.tsv", sep="\t", index=False)

### Create Subset

In [9]:
chembl_np = pd.read_csv("../tmp/chembl_np_std.tsv", sep="\t")
chembl_np_subset = chembl_np.sample(n=5000, random_state=0xc0ffee)
chembl_np_subset.to_csv("../tmp/chembl_np_std_subset.tsv", sep="\t", index=False)
lp(chembl_np_subset, "ChEMBL NP subset")

Shape ChEMBL NP subset                            :       5000 /    3  [ Name, chembl_id, Smiles ]  


## DrugBank

Approved and experimental drugs, downloaded from DrugBank on 25-Mar-2020, v 5.1.5. (https://www.drugbank.com/)

* 8479 cpds.
* 7472 cpds. after standardization and deduplication

### Loading

Load both SD files of approved and experimental drugs.

In [3]:
drugbank = u.read_sdf(["/home/pahl/comas/notebooks/sdf/drugbank/drugbank_5.1.5_appr.sdf", "/home/pahl/comas/notebooks/sdf/drugbank/drugbank_5.1.5_exp.sdf"])
drugbank =drugbank[["DATABASE_ID", "Smiles"]]
lp(drugbank, "DrugBank")

drugbank.head()

RDKit ERROR: [21:16:13] ERROR: Explicit valence for atom # 6 K, 2, is greater than permitted
RDKit ERROR: [21:17:24] Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [21:17:24] ERROR: Could not sanitize molecule ending on line 19217
RDKit ERROR: [21:17:24] ERROR: Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [21:17:25] Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [21:17:25] ERROR: Could not sanitize molecule ending on line 318021
RDKit WARNING: [21:17:26] MRV_IMPLICIT_H SGroup on atom without aromatic bonds, 6, ignored.
RDKit WARNING: [21:17:26] MRV_IMPLICIT_H SGroup on atom without aromatic bonds, 11, ignored.
RDKit WARNING: [21:17:26] MRV_IMPLICIT_H SGroup on atom without aromatic bonds, 10, ignored.
RDKit WARNING: [21:17:26] MRV_IMPLICIT_H SGroup on atom without aromatic bonds, 14, ignored.
RDKit ERROR: [21:17:25] ERROR: Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [2

{'In': 8479, 'Out': 8462, 'Fail_NoMol': 17}
Shape DrugBank                                    :       8462 /    2  [ DATABASE_ID, Smiles ]  


RDKit WARNING: [21:17:30] MRV_IMPLICIT_H SGroup on atom without aromatic bonds, 6, ignored.
RDKit WARNING: [21:17:30] MRV_IMPLICIT_H SGroup on atom without aromatic bonds, 11, ignored.
RDKit WARNING: [21:17:30] MRV_IMPLICIT_H SGroup on atom without aromatic bonds, 10, ignored.
RDKit WARNING: [21:17:30] MRV_IMPLICIT_H SGroup on atom without aromatic bonds, 14, ignored.
RDKit ERROR: [21:17:29] ERROR: Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [21:17:30] Explicit valence for atom # 14 Cl, 2, is greater than permitted
RDKit ERROR: [21:17:30] ERROR: Could not sanitize molecule ending on line 925357
RDKit ERROR: [21:17:30] ERROR: Explicit valence for atom # 14 Cl, 2, is greater than permitted
RDKit ERROR: [21:17:30] Explicit valence for atom # 6 K, 2, is greater than permitted
RDKit ERROR: [21:17:30] ERROR: Could not sanitize molecule ending on line 930170


,DATABASE_ID,Smiles
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(=O)O)NC(=O)[C@...
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(=N)N)NC(=...
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
3,DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
4,DB00035,N=C(N)NCCC[C@@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H]...


### Standardization

Standardization implemented in MolVS:
* standardization
* remove stereo information
* choose largest fragment
* canonicalize tautomer

In [4]:
drugbank = u.apply_to_smiles(drugbank, "Smiles", {"Smiles_Std": lambda x: u.standardize_mol(x, remove_stereo=True, canonicalize_tautomer=True)})
drugbank.drop("Smiles", axis=1, inplace=True)
drugbank = drugbank.rename(columns={"Smiles_Std": "Smiles"})
lp(drugbank, "DrugBank")

RDKit ERROR: [21:17:30] ERROR: Explicit valence for atom # 6 K, 2, is greater than permitted
RDKit ERROR: [21:26:09] Can't kekulize mol.  Unkekulized atoms: 3 10
RDKit ERROR: 
RDKit ERROR: [21:26:09] Can't kekulize mol.  Unkekulized atoms: 3 10
RDKit ERROR: 
RDKit ERROR: [21:28:07] Explicit valence for atom # 6 B, 5, is greater than permitted
RDKit ERROR: [21:28:56] Explicit valence for atom # 13 B, 5, is greater than permitted
RDKit ERROR: [21:29:07] Explicit valence for atom # 16 B, 5, is greater than permitted
RDKit ERROR: [21:30:01] Explicit valence for atom # 16 B, 5, is greater than permitted
RDKit ERROR: [21:30:27] Explicit valence for atom # 13 B, 5, is greater than permitted
RDKit ERROR: [21:30:39] Explicit valence for atom # 6 B, 5, is greater than permitted
RDKit ERROR: [21:31:56] Explicit valence for atom # 6 B, 5, is greater than permitted
RDKit ERROR: [21:34:05] Explicit valence for atom # 16 B, 5, is greater than permitted
RDKit ERROR: [21:34:05] Explicit valence for ato


Shape DrugBank                                    :       8462 /    2  [ DATABASE_ID, Smiles ]  ( NAN values in 1 col(s) )


### Filters (Isotopes, MedChem, NumHeavyAtoms, Duplicates)

The data set is filtered for Isotopes, MedChemAtoms, MinHeavyAtoms, MaxHeavyAtoms and Duplicates.  

In [5]:
drugbank = u.filter_mols(drugbank, "Smiles", ["Isotopes", "MedChemAtoms", "MinHeavyAtoms", "MaxHeavyAtoms", "Duplicates"])
lp(drugbank, "DrugBank (MedChem filters)")

drugbank.head(10)

Applying filters (5)...



Applied filter Isotopes: 8404



Applied filter MedChemAtoms: 8187



Applied filter MinHeavyAtoms: 8142
Applied filter MaxHeavyAtoms: 8058



Applied filter Duplicates: 7472
Shape DrugBank (MedChem filters)                  :       7472 /    2  [ DATABASE_ID, Smiles ]  


,DATABASE_ID,Smiles
4,DB00035,N=C(N)NCCCC(NC(=O)C1CCCN1C(=O)C1CSSCCC(=O)NC(C...
8,DB00104,CC(O)C(CO)NC(=O)C1CSSCC(NC(=O)C(N)Cc2ccccc2)C(...
10,DB00114,Cc1ncc(CO[PH](=O)(=O)O)c(C=O)c1O
12,DB00118,C[S+](CCC(N)C(=O)[O-])CC1OC(n2cnc3c(=N)[nH]cnc...
13,DB00119,CC(=O)C(=O)O
14,DB00120,NC(Cc1ccccc1)C(=O)O
15,DB00121,O=C(O)CCCCC1SCC2NC(=O)NC21
16,DB00122,C[N+](C)(C)CCO
17,DB00123,NCCCCC(N)C(=O)O
18,DB00126,OCC(O)c1oc(O)c(O)c1O


### Write Data Set

In [8]:
drugbank.to_csv("../tmp/drugbank_std.tsv", sep="\t", index=False)
lp(drugbank, "DrugBank (MedChem filters)")

drugbank.head()

Shape DrugBank (MedChem filters)                  :       7472 /    2  [ DATABASE_ID, Smiles ]  


,DATABASE_ID,Smiles
4,DB00035,N=C(N)NCCCC(NC(=O)C1CCCN1C(=O)C1CSSCCC(=O)NC(C...
8,DB00104,CC(O)C(CO)NC(=O)C1CSSCC(NC(=O)C(N)Cc2ccccc2)C(...
10,DB00114,Cc1ncc(CO[PH](=O)(=O)O)c(C=O)c1O
12,DB00118,C[S+](CCC(N)C(=O)[O-])CC1OC(n2cnc3c(=N)[nH]cnc...
13,DB00119,CC(=O)C(=O)O


### Create Subset

In [13]:
drugbank = pd.read_csv("../tmp/drugbank_std.tsv", sep="\t")
drugbank_subset = drugbank.sample(n=5000, random_state=0xc0ffee)
drugbank_subset.to_csv("../tmp/drugbank_std_subset.tsv", sep="\t", index=False)
lp(drugbank_subset, "Drugbank subset")

Shape Drugbank subset                             :       5000 /    2  [ DATABASE_ID, Smiles ]  


## Enamine Advanced Screening Collection

* Enamine Advanced Screening Collection (527411 compounds)
* Downloaded from [Enamine](https://enamine.net/hit-finding/compound-collections/screening-collection/advanced-collection) on 07-Dec-2020
SD file extracted and gzipped
Random selection of 50k, then standardized and filtered.

### Loading

Load the downloaded gzipped SD file into a DataFrame

In [3]:
enamine = u.read_sdf(["/home/pahl/comas/notebooks/sdf/enamine/enamine_adv.gz"])
enamine = enamine[["idnumber", "Smiles"]]
lp(enamine, "Enamine")

enamine.head()

RDKit ERROR: [17:22:53] Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [17:22:53] ERROR: Could not sanitize molecule ending on line 24061869
RDKit ERROR: [17:22:53] ERROR: Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [17:22:53] Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [17:22:53] ERROR: Could not sanitize molecule ending on line 24061948
RDKit ERROR: [17:22:53] ERROR: Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [17:23:13] Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [17:23:13] ERROR: Could not sanitize molecule ending on line 28077193
RDKit ERROR: [17:23:13] ERROR: Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [17:23:35] Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [17:23:35] ERROR: Could not sanitize molecule ending on line 32668058
RDKit ERROR: [17:23:35] ERROR: Explicit valen

{'In': 527411, 'Out': 527380, 'Fail_NoMol': 31}
Shape Enamine                                     :     527380 /    2  [ idnumber, Smiles ]  


,idnumber,Smiles
0,Z3234812272,C[C@@H](N)CN.Cl.Cl
1,Z1198156558,c1nncs1
2,Z2941405027,CC(F)CN.Cl
3,Z2941400887,Cl.NCC=CCl
4,Z1255360409,O=P(O)(O)F


### Create Subset

A random subset of 50k cpds. is selected.

In [4]:
enamine_subset = enamine.sample(n=50000, random_state=0xc0ffee)
enamine_subset.to_csv("../tmp/enamine_subset.tsv", sep="\t", index=False)

In [3]:
# enamine_subset = pd.read_csv("../tmp/enamine_subset.tsv", sep="\t")

### Standardization

Standardization implemented in MolVS:
* standardization
* remove stereo information
* choose largest fragment
* canonicalize tautomer

In [4]:
enamine_subset = u.apply_to_smiles(enamine_subset, "Smiles", {"Smiles_Std": lambda x: u.standardize_mol(x, remove_stereo=True, canonicalize_tautomer=True)}, parallel=True)
enamine_subset.drop("Smiles", axis=1, inplace=True)
enamine_subset = enamine_subset.rename(columns={"Smiles_Std": "Smiles"})
lp(enamine_subset, "Enamine (Std)")

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


RDKit ERROR: [17:35:45] Explicit valence for atom # 24 B, 5, is greater than permitted


Shape Enamine (Std)                               :      50000 /    2  [ idnumber, Smiles ]  ( NAN values in 1 col(s) )


### Filters

The data set is filtered for Isotopes, MedChemAtoms, MinHeavyAtoms, MaxHeavyAtoms and Duplicates.  

In [5]:
enamine_subset = u.filter_mols(enamine_subset, "Smiles", ["Isotopes", "MedChemAtoms", "MinHeavyAtoms", "MaxHeavyAtoms", "Duplicates"])
lp(enamine_subset, "Enamine (MedChem filters)")

enamine_subset.head(10)

Applying filters (5)...



Applied filter Isotopes: 49999



Applied filter MedChemAtoms: 49992



Applied filter MinHeavyAtoms: 49992
Applied filter MaxHeavyAtoms: 49991



Applied filter Duplicates: 49968
Shape Enamine (MedChem filters)                   :      49968 /    2  [ idnumber, Smiles ]  


,idnumber,Smiles
0,Z2970969541,CCOc1cc(N2CCN(C(N)=O)CC2)nc(C)n1
1,Z2391619370,O=C(CCc1ccco1)N1CCCC(O)(Cc2nc(C3CC3)no2)C1
2,Z1440436506,Cc1nc(CC(C)C)c(C(=O)N2CC(O)C2)s1
3,Z1334208338,Cc1cc(C(=O)NCCc2nnc(-c3ccccc3)o2)n(C)n1
4,Z1396752281,Fc1cccnc1N1CCC(N=c2ncc(Br)c[nH]2)C1
5,Z1759227827,COc1cc(C(=O)NCC2CC(F)CN2Cc2cnoc2C)on1
6,Z2755041138,CC(C)(C)OC(=O)N1CCN(C(C(=O)NCC(N)=O)C2CCCC2)CC1
7,Z1704446318,CC1CCN(C(O)c2cc(=O)[nH]c3ccccc23)c2c(O)cccc21
8,Z3484536734,O=c1[nH]c(=O)c2c(-c3ccc(OCCCN4CCCCC4)cc3)[nH]n...
9,Z1693137190,CCCCC(NC(=O)c1ccc(N2CCCS2(=O)=O)cc1C)C(=O)OC(C...


In [6]:
enamine_subset.to_csv("../tmp/enamine_subset_std.tsv", sep="\t", index=False)
enamine_subset[["idnumber"]].to_csv("../Input Data/enamine_subset_std_ids.tsv", sep="\t", index=False)
lp(enamine_subset, "Enamine (filters)")


Shape Enamine (filters)                           :      49968 /    2  [ idnumber, Smiles ]  


,idnumber,Smiles
0,Z2970969541,CCOc1cc(N2CCN(C(N)=O)CC2)nc(C)n1
1,Z2391619370,O=C(CCc1ccco1)N1CCCC(O)(Cc2nc(C3CC3)no2)C1
2,Z1440436506,Cc1nc(CC(C)C)c(C(=O)N2CC(O)C2)s1
3,Z1334208338,Cc1cc(C(=O)NCCc2nnc(-c3ccccc3)o2)n(C)n1
4,Z1396752281,Fc1cccnc1N1CCC(N=c2ncc(Br)c[nH]2)C1


## Internal Compounds

Approved and experimental drugs, downloaded from DrugBank on 25-Mar-2020, v 5.1.5.

* 244 individual cpds.
* standardization, removal of stereo information and MedChem filters: `internal_cpds_mc.tsv`
    * this data set is used for the chemical class-similarity calculations
* deduplication, based on InChIKeys: `internal_cpds_std.tsv` (168 cpds.)
    * analogous workflow to the reference data sets, without deglycosylation
    * this data set is used for the NP Likeness calculations and the PCA analysis

### Loading

Load Load list of 244 compounds with their Smiles.

In [3]:
cpds = pd.read_csv("../Input Data/internal_cpds.tsv", sep="\t")
lp(cpds, "Internal compounds")

cpds.head()

Shape Internal compounds                          :        244 /    3  [ Compound_Id, CpdClass, Smiles ]  


,Compound_Id,CpdClass,Smiles
0,408580,QN-C-S,COc1ccc2nccc([C@@H](O)[C@@H]3C[C@@H]4CCN3C[C@]...
1,408614,QN-C-S,COc1ccc2nccc([C@@H](O)[C@@H]3C[C@@H]4CCN3C[C@]...
2,409679,QN-C-S,COc1ccc2nccc([C@@H](O)[C@@H]3C[C@@H]4CCN3C[C@]...
3,408605,QN-C-S,COc1ccc2nccc([C@@H](O)[C@@H]3C[C@@H]4CCN3C[C@]...
4,408610,QN-C-S,COc1ccc2c(c1)O[C@]1(CC2=O)CN2CC[C@H]1C[C@H]2[C...


### Standardization

Standardization implemented in MolVS:
* standardization
* remove stereo information
* choose largest fragment
* canonicalize tautomer

In [4]:
cpds = u.apply_to_smiles(cpds, "Smiles", {"Smiles_Std": lambda x: u.standardize_mol(x, remove_stereo=True, canonicalize_tautomer=True)})
cpds.drop("Smiles", axis=1, inplace=True)
cpds = cpds.rename(columns={"Smiles_Std": "Smiles"})
lp(cpds, "Internal compounds")


Shape Internal compounds                          :        244 /    3  [ Compound_Id, CpdClass, Smiles ]  


### Filters (MedChem)

The data set is filtered for Isotopes, MedChemAtoms, MinHeavyAtoms, MaxHeavyAtoms.  

In [5]:
cpds = u.filter_mols(cpds, "Smiles", ["Isotopes", "MedChemAtoms", "MinHeavyAtoms", "MaxHeavyAtoms"])
lp(cpds, "Internal compounds (MedChem filters)")

cpds.head(10)

Applying filters (4)...



Applied filter Isotopes: 244



Applied filter MedChemAtoms: 244



Applied filter MinHeavyAtoms: 244
Applied filter MaxHeavyAtoms: 244
Shape Internal compounds (MedChem filters)        :        244 /    3  [ Compound_Id, CpdClass, Smiles ]  


,Compound_Id,CpdClass,Smiles
0,408580,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4ccccc4O3...
1,408614,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(Br)cc...
2,409679,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(Cl)cc...
3,408605,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(F)ccc...
4,408610,QN-C-S,COc1ccc2c(c1)OC1(CC2=O)CN2CCC1CC2C(O)c1ccnc2cc...
5,408643,QN-C-S,COc1cc(OC)c2c(c1)OC1(CC2=O)CN2CCC1CC2C(O)c1ccn...
6,408636,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4ccc(OC)c...
7,408584,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4c(O)cccc...
8,408586,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4ccc(O)cc...
9,408612,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(C)ccc...


In [6]:
cpds.to_csv("../tmp/internal_cpds_mc.tsv", sep="\t", index=False)
lp(cpds, "Internal compounds (MedChem filters)")

cpds.head()

Shape Internal compounds (MedChem filters)        :        244 /    3  [ Compound_Id, CpdClass, Smiles ]  


,Compound_Id,CpdClass,Smiles
0,408580,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4ccccc4O3...
1,408614,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(Br)cc...
2,409679,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(Cl)cc...
3,408605,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(F)ccc...
4,408610,QN-C-S,COc1ccc2c(c1)OC1(CC2=O)CN2CCC1CC2C(O)c1ccnc2cc...


### Filters (Duplicates)

The data set is filtered for Duplicates (based on InChIKeys (the stereo centers have been removed in the previous standardization step)).  
This is done for consistency with the standardization and filtering of the reference sets.  
This data set is used for the NP Likeness calculations and the PCA analysis.

In [7]:
cpds = u.filter_mols(cpds, "Smiles", ["Duplicates"])
lp(cpds, "Internal compounds (Duplicates)")

cpds.head(10)

Applying filters (1)...



Applied filter Duplicates: 168
Shape Internal compounds (Duplicates)             :        168 /    3  [ Compound_Id, CpdClass, Smiles ]  


,Compound_Id,CpdClass,Smiles
0,408580,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4ccccc4O3...
1,408614,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(Br)cc...
2,409679,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(Cl)cc...
3,408605,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(F)ccc...
4,408610,QN-C-S,COc1ccc2c(c1)OC1(CC2=O)CN2CCC1CC2C(O)c1ccnc2cc...
5,408643,QN-C-S,COc1cc(OC)c2c(c1)OC1(CC2=O)CN2CCC1CC2C(O)c1ccn...
6,408636,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4ccc(OC)c...
7,408584,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4c(O)cccc...
8,408586,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4ccc(O)cc...
9,408612,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(C)ccc...


In [8]:
cpds.to_csv("../tmp/internal_cpds_std.tsv", sep="\t", index=False)
lp(cpds, "Internal compounds (MedChem filters)")

cpds.head()

Shape Internal compounds (MedChem filters)        :        168 /    3  [ Compound_Id, CpdClass, Smiles ]  


,Compound_Id,CpdClass,Smiles
0,408580,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4ccccc4O3...
1,408614,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(Br)cc...
2,409679,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(Cl)cc...
3,408605,QN-C-S,COc1ccc2nccc(C(O)C3CC4CCN3CC43CC(=O)c4cc(F)ccc...
4,408610,QN-C-S,COc1ccc2c(c1)OC1(CC2=O)CN2CCC1CC2C(O)c1ccnc2cc...


In [9]:
!cp ../tmp/internal_cpds_mc.tsv "../Input Data/internal_cpds_mc.tsv"
!cp ../tmp/internal_cpds_std.tsv "../Input Data/internal_cpds_std.tsv"